# Task 2 of Data visualization using python

## Creating a Dashboard using python

### Dashboard for visualizing sales data ,i.e historical-automobile-sales using historical-automobile-sales data of XYZAutomobiles


In [ ]:
!pip install setuptools
!pip install packaging
!pip install pandas dash
!pip install more-itertools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 36.7 MB/s eta 0:00:00


In [10]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load data
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# Set the title of the dashboard
app.title = "Automobile Statistics Dashboard"

# Create the dropdown menu options
dropdown_options = [
    {'label': 'Non-Recession Period statistics', 'value': 'Yearly Statistics'},
    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
]

# List of years
year_list = [i for i in range(1980, 2024, 1)]

# Create the layout of the app
app.layout = html.Div([
    html.H1("Automobile Sales Statistics Dashboard", style={'text-align': 'center', 'margin-bottom': '20px'}),

    html.Div([
        html.Label("Select Statistics:", style={'margin-right': '10px'}),
        dcc.Dropdown(
            id='dropdown-statistics',
            options=dropdown_options,
            value='Yearly Statistics',
            placeholder='Select a report type',
            style={'width': '300px', 'display': 'inline-block'}
        )
    ], style={'text-align': 'center', 'margin-bottom': '20px'}),

    html.Div([
        dcc.Dropdown(
            id='select-year',
            options=[{'label': i, 'value': i} for i in year_list],
            value=year_list[0],
            placeholder='Select a year',
            style={'width': '200px', 'display': 'inline-block'}
        )
    ], style={'text-align': 'center', 'margin-bottom': '20px'}),

    html.Div([
        html.Div(id='output-container', className='chart-grid', style={'display': 'flex', 'flex-wrap': 'wrap', 'justify-content': 'center'}),
    ])
], style={'margin': '20px'})

@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics', component_property='value')
)
def update_input_container(selected_statistics):
    if selected_statistics == 'Yearly Statistics':
        return False
    else:
        return True

@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics', component_property='value'),
     Input(component_id='select-year', component_property='value')]
)
def update_output_container(selected_statistics, input_year):
    if selected_statistics == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = data[data['Recession'] == 1]

        # Plot 1: Automobile sales fluctuate over Recession Period (year-wise)
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec,
                           x='Year',
                           y='Automobile_Sales',
                           title="Avg Automobile Sales Fluctuation over Recession",
                           template='plotly_white').update_layout(height=300)
        )

        # Plot 2: Average number of vehicles sold by vehicle type
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(
            figure=px.bar(average_sales,
                          x='Vehicle_Type',
                          y='Automobile_Sales',
                          title="Avg Vehicles Sold by Type",
                          template='plotly_white').update_layout(height=300)
        )

        # Plot 3: Total expenditure share by vehicle type during recessions
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(exp_rec,
                          values='Advertising_Expenditure',
                          names='Vehicle_Type',
                          title="Expenditure Share by Vehicle Type",
                          template='plotly_white').update_layout(height=300)
        )

        # Plot 4: Effect of unemployment rate on vehicle type and sales
        unemp_data = recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(
            figure=px.bar(unemp_data,
                          x='unemployment_rate',
                          y='Automobile_Sales',
                          color='Vehicle_Type',
                          labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Avg Automobile Sales'},
                          title='Effect of Unemployment on Sales',
                          template='plotly_white').update_layout(height=300)
        )

        return [
            html.Div(className='chart-item', children=[R_chart1], style={'width': '45%', 'padding': '5px'}),
            html.Div(className='chart-item', children=[R_chart2], style={'width': '45%', 'padding': '5px'}),
            html.Div(className='chart-item', children=[R_chart3], style={'width': '45%', 'padding': '5px'}),
            html.Div(className='chart-item', children=[R_chart4], style={'width': '45%', 'padding': '5px'})
        ]

    elif selected_statistics == 'Yearly Statistics':
        yearly_data = data[data['Year'] == input_year]

        # Plot 1: Yearly Automobile sales using line chart for the whole period
        yas = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(yas,
                           x='Year',
                           y='Automobile_Sales',
                           title='Yearly Automobile Sales',
                           template='plotly_white').update_layout(height=300)
        )

        # Plot 2: Total Monthly Automobile sales using line chart
        mas = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(
            figure=px.line(mas,
                           x='Month',
                           y='Automobile_Sales',
                           title='Monthly Automobile Sales',
                           template='plotly_white').update_layout(height=300)
        )

        # Plot 3: Average number of vehicles sold during the given year
        avr_vdata = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(
            figure=px.bar(avr_vdata,
                          x='Vehicle_Type',
                          y='Automobile_Sales',
                          title=f'Avg Vehicles Sold by Type in {input_year}',
                          template='plotly_white').update_layout(height=300)
        )

        # Plot 4: Total Advertisement Expenditure for each vehicle using pie chart
        exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(exp_data,
                          names='Vehicle_Type',
                          values='Advertising_Expenditure',
                          title='Ad Expenditure by Vehicle Type',
                          template='plotly_white').update_layout(height=300)
        )

        return [
            html.Div(className='chart-item', children=[Y_chart1], style={'width': '45%', 'padding': '5px'}),
            html.Div(className='chart-item', children=[Y_chart2], style={'width': '45%', 'padding': '5px'}),
            html.Div(className='chart-item', children=[Y_chart3], style={'width': '45%', 'padding': '5px'}),
            html.Div(className='chart-item', children=[Y_chart4], style={'width': '45%', 'padding': '5px'})
        ]

    else:
        return None

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>